# Libraries

In [7]:
import json
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from textblob import TextBlob
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from textblob import TextBlob
import os
from autocorrect import Speller

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Load JSON Files

In [8]:
# English Data
data_en = json.loads(open('data_en.json').read())

# Spanish Data
data_es = json.loads(open('data_es.json').read())

## Natural Language Processing

In [10]:
intents_en = data_en['interactionModel']['languageModel']

intents_es = data_es['interactionModel']['languageModel']


In [15]:
# Create a list of tuples that assigns a list of words to an intent in English
words_en = []
labels_en = []
documents_en = []
punc_en = ['?', '!', ',', '.']

for intent in intents_en['intent']: 
    for sample in intent['samples']:
        word_list_en = nltk.word_tokenize(sample)
        words_en.extend(word_list_en)
        documents_en.append((word_list_en, intent['name']))
        if intent['name'] not in labels_en:
            labels_en.append(intent['name'])


In [18]:
# Create a list of tuples that assigns a list of words to an intent in Spanish
words_es = []
labels_es = []
documents_es = []
punc_es = ['?', '!', ',', '.']

for intent in intents_es['intent']: 
    for sample in intent['samples']:
        word_list_es = nltk.word_tokenize(sample)
        words_es.extend(word_list_es)
        documents_es.append((word_list_es, intent['name']))
        if intent['name'] not in labels_es:
            labels_es.append(intent['name'])


## Bag of Words

In [19]:
# import NLP library
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

In [21]:
# turn words to lower case and to base form
words_en = [lem.lemmatize(word.lower()) for word in words_en if word not in punc_en]
words_es = [lem.lemmatize(word.lower()) for word in words_es if word not in punc_es]


# sort words and remove duplicates in English
words_en = sorted(set(words_en))
labels_en = sorted(set(labels_en))

# sort words and remove duplicates in Spanish
words_es = sorted(set(words_es))
labels_es = sorted(set(labels_es))


## Preprocessing input and output data 

In [23]:
# create training and output arrays in English
training_en = []
output_empty_en = [0] * len(labels_en)
for document in documents_en:
    bag_en = []
    word_patterns_en = document[0]
    word_patterns_en = [lem.lemmatize(word.lower()) for word in word_patterns_en]
    for word in words_en:
        if word in word_patterns_en:
            bag_en.append(1)
        else:
            bag_en.append(0)
            
    # copy output list
    output_en = list(output_empty_en)
    output_en[labels_en.index(document[1])] = 1
    training_en.append([bag_en, output_en])


In [28]:
# create training and output arrays in Spanish
training_es = []
output_empty_es = [0] * len(labels_es)
for document in documents_es:
    bag_es = []
    word_patterns_es = document[0]
    word_patterns_es = [lem.lemmatize(word.lower()) for word in word_patterns_es]
    for word in words_es:
        if word in word_patterns_es:
            bag_es.append(1)
        else:
            bag_es.append(0)
            
    # copy output list
    output_es = list(output_empty_es)
    output_es[labels_es.index(document[1])] = 1
    training_es.append([bag_es, output_es])


In [30]:
# randomizing the order of the training data in English
random.shuffle(training_en)
training_en = np.array(training_en)

train_x_en = list(training_en[:,0])
train_y_en = list(training_en[:,1])

# randomizing the order of the training data in Spanish
random.shuffle(training_es)
training_es = np.array(training_es)

train_x_es = list(training_es[:,0])
train_y_es = list(training_es[:,1])


C:\Users\aashi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


## Build Model

In [35]:
def model_en():
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x_en[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y_en[0]), activation='softmax'))
    return model


def model_es():
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x_es[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y_es[0]), activation='softmax'))
    return model


### Train English Model 

In [34]:
model_en = model_en()

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_en.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_en = ModelCheckpoint(filepath='model_en.h5', monitor='acc')
model_en.fit(np.array(train_x_en), np.array(train_y_en), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint_en])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
91/91 [==============================] - 0s 3ms/sample - loss: 3.0888 - acc: 0.0330
Epoch 2/500
91/91 [==============================] - 0s 285us/sample - loss: 2.8313 - acc: 0.1758
Epoch 3/500
91/91 [==============================] - 0s 285us/sample - loss: 2.6667 - acc: 0.2308
Epoch 4/500
91/91 [==============================] - 0s 318us/sample - loss: 2.4801 - acc: 0.3077
Epoch 5/500
91/91 [==============================] - 0s 329us/sample - loss: 2.2401 - acc: 0.3736
Epoch 6/500
91/91 [==============================] - 0s 318us/sample - loss: 2.1401 - acc: 0.3516
Epoch 7/500
91/91 [==============================] - 0s 329us/sample - loss: 2.0474 - acc: 0.3846
Epoch 8/500
91/91 [==============================] - 0s 318us/sample - loss: 1.80

91/91 [==============================] - 0s 296us/sample - loss: 0.1018 - acc: 0.9670
Epoch 75/500
91/91 [==============================] - 0s 307us/sample - loss: 0.1201 - acc: 0.9670
Epoch 76/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0610 - acc: 0.9890
Epoch 77/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1244 - acc: 0.9670
Epoch 78/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0671 - acc: 0.9890
Epoch 79/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1128 - acc: 0.9451
Epoch 80/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0962 - acc: 0.9780
Epoch 81/500
91/91 [==============================] - 0s 274us/sample - loss: 0.1358 - acc: 0.9560
Epoch 82/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0915 - acc: 0.9670
Epoch 83/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0928 - acc: 0.9780
Epoch 84/500
91/91 [===

91/91 [==============================] - 0s 252us/sample - loss: 0.0255 - acc: 1.0000
Epoch 157/500
91/91 [==============================] - 0s 296us/sample - loss: 0.0509 - acc: 1.0000
Epoch 158/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0358 - acc: 0.9890
Epoch 159/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0287 - acc: 1.0000
Epoch 160/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0399 - acc: 0.9890
Epoch 161/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0178 - acc: 1.0000
Epoch 162/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0225 - acc: 1.0000
Epoch 163/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0618 - acc: 0.9780
Epoch 164/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0303 - acc: 1.0000
Epoch 165/500
91/91 [==============================] - 0s 318us/sample - loss: 0.0348 - acc: 1.0000
Epoch 166/500


91/91 [==============================] - 0s 263us/sample - loss: 0.1190 - acc: 0.9670
Epoch 239/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0825 - acc: 0.9670
Epoch 240/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0807 - acc: 0.9670
Epoch 241/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0248 - acc: 1.0000
Epoch 242/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0473 - acc: 0.9890
Epoch 243/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0714 - acc: 0.9780
Epoch 244/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0717 - acc: 0.9780
Epoch 245/500
91/91 [==============================] - 0s 274us/sample - loss: 0.1692 - acc: 0.9451
Epoch 246/500
91/91 [==============================] - 0s 296us/sample - loss: 0.0391 - acc: 0.9780
Epoch 247/500
91/91 [==============================] - 0s 307us/sample - loss: 0.0244 - acc: 1.0000
Epoch 248/500


91/91 [==============================] - 0s 252us/sample - loss: 0.0283 - acc: 0.9890
Epoch 321/500
91/91 [==============================] - 0s 296us/sample - loss: 0.0119 - acc: 1.0000
Epoch 322/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0679 - acc: 0.9780
Epoch 323/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0235 - acc: 1.0000
Epoch 324/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0167 - acc: 1.0000
Epoch 325/500
91/91 [==============================] - 0s 241us/sample - loss: 0.0327 - acc: 0.9780
Epoch 326/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0337 - acc: 0.9890
Epoch 327/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0062 - acc: 1.0000
Epoch 328/500
91/91 [==============================] - 0s 307us/sample - loss: 0.0097 - acc: 1.0000
Epoch 329/500
91/91 [==============================] - 0s 340us/sample - loss: 0.0159 - acc: 1.0000
Epoch 330/500


91/91 [==============================] - 0s 285us/sample - loss: 0.0222 - acc: 0.9890
Epoch 403/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0382 - acc: 0.9890
Epoch 404/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0234 - acc: 0.9890
Epoch 405/500
91/91 [==============================] - 0s 285us/sample - loss: 0.0410 - acc: 0.9890
Epoch 406/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0060 - acc: 1.0000
Epoch 407/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0226 - acc: 1.0000
Epoch 408/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0155 - acc: 0.9890
Epoch 409/500
91/91 [==============================] - 0s 296us/sample - loss: 0.0115 - acc: 1.0000
Epoch 410/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0043 - acc: 1.0000
Epoch 411/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0352 - acc: 0.9890
Epoch 412/500


91/91 [==============================] - 0s 252us/sample - loss: 0.0196 - acc: 1.0000
Epoch 485/500
91/91 [==============================] - 0s 307us/sample - loss: 0.0159 - acc: 0.9890
Epoch 486/500
91/91 [==============================] - 0s 296us/sample - loss: 0.0119 - acc: 1.0000
Epoch 487/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0242 - acc: 0.9890
Epoch 488/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0076 - acc: 1.0000
Epoch 489/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0203 - acc: 0.9890
Epoch 490/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0458 - acc: 0.9780
Epoch 491/500
91/91 [==============================] - ETA: 0s - loss: 0.0981 - acc: 1.000 - 0s 263us/sample - loss: 0.0138 - acc: 1.0000
Epoch 492/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0140 - acc: 1.0000
Epoch 493/500
91/91 [==============================] - 0s 296us/sample - los

### Train Spanish Model 

In [36]:
model_es = model_es()

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_es.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_es = ModelCheckpoint(filepath='model_es.h5', monitor='acc')
model_es.fit(np.array(train_x_es), np.array(train_y_es), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint_es])

Epoch 1/500
52/52 [==============================] - 0s 4ms/sample - loss: 3.1043 - acc: 0.0192
Epoch 2/500
52/52 [==============================] - 0s 403us/sample - loss: 3.0831 - acc: 0.1154
Epoch 3/500
52/52 [==============================] - 0s 384us/sample - loss: 2.9848 - acc: 0.0962
Epoch 4/500
52/52 [==============================] - 0s 384us/sample - loss: 2.9483 - acc: 0.1346
Epoch 5/500
52/52 [==============================] - 0s 441us/sample - loss: 2.8862 - acc: 0.1731
Epoch 6/500
52/52 [==============================] - 0s 441us/sample - loss: 2.7410 - acc: 0.2692
Epoch 7/500
52/52 [==============================] - 0s 441us/sample - loss: 2.7159 - acc: 0.2115
Epoch 8/500
52/52 [==============================] - 0s 441us/sample - loss: 2.6915 - acc: 0.2692
Epoch 9/500
52/52 [==============================] - 0s 499us/sample - loss: 2.6166 - acc: 0.2692
Epoch 10/500
52/52 [==============================] - 0s 422us/sample - loss: 2.4515 - acc: 0.2692
Epoch 11/500
52/52 [=

52/52 [==============================] - 0s 403us/sample - loss: 0.2208 - acc: 0.9808
Epoch 84/500
52/52 [==============================] - 0s 384us/sample - loss: 0.3342 - acc: 0.9038
Epoch 85/500
52/52 [==============================] - 0s 384us/sample - loss: 0.3007 - acc: 0.9038
Epoch 86/500
52/52 [==============================] - 0s 384us/sample - loss: 0.2773 - acc: 0.9808
Epoch 87/500
52/52 [==============================] - 0s 364us/sample - loss: 0.3235 - acc: 0.9231
Epoch 88/500
52/52 [==============================] - 0s 365us/sample - loss: 0.4332 - acc: 0.8269
Epoch 89/500
52/52 [==============================] - 0s 384us/sample - loss: 0.1625 - acc: 0.9615
Epoch 90/500
52/52 [==============================] - 0s 384us/sample - loss: 0.3401 - acc: 0.9038
Epoch 91/500
52/52 [==============================] - 0s 345us/sample - loss: 0.2994 - acc: 0.9231
Epoch 92/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1561 - acc: 0.9615
Epoch 93/500
52/52 [===

52/52 [==============================] - 0s 422us/sample - loss: 0.1276 - acc: 0.9808
Epoch 166/500
52/52 [==============================] - 0s 365us/sample - loss: 0.0892 - acc: 0.9808
Epoch 167/500
52/52 [==============================] - 0s 384us/sample - loss: 0.1037 - acc: 0.9808
Epoch 168/500
52/52 [==============================] - 0s 384us/sample - loss: 0.1065 - acc: 0.9615
Epoch 169/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1190 - acc: 0.9615
Epoch 170/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1133 - acc: 0.9808
Epoch 171/500
52/52 [==============================] - 0s 365us/sample - loss: 0.1166 - acc: 0.9423
Epoch 172/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0596 - acc: 1.0000
Epoch 173/500
52/52 [==============================] - 0s 345us/sample - loss: 0.1190 - acc: 0.9808
Epoch 174/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0286 - acc: 1.0000
Epoch 175/500


52/52 [==============================] - 0s 364us/sample - loss: 0.0880 - acc: 0.9615
Epoch 248/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1173 - acc: 0.9423
Epoch 249/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0113 - acc: 1.0000
Epoch 250/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0940 - acc: 0.9808
Epoch 251/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1265 - acc: 0.9615
Epoch 252/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0753 - acc: 0.9808
Epoch 253/500
52/52 [==============================] - 0s 422us/sample - loss: 0.1439 - acc: 0.9615
Epoch 254/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0890 - acc: 0.9808
Epoch 255/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0293 - acc: 1.0000
Epoch 256/500
52/52 [==============================] - 0s 326us/sample - loss: 0.0694 - acc: 0.9615
Epoch 257/500


52/52 [==============================] - 0s 403us/sample - loss: 0.0168 - acc: 1.0000
Epoch 329/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0448 - acc: 1.0000
Epoch 330/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0214 - acc: 1.0000
Epoch 331/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0211 - acc: 1.0000
Epoch 332/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0516 - acc: 0.9808
Epoch 333/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0239 - acc: 1.0000
Epoch 334/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0441 - acc: 0.9808
Epoch 335/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1391 - acc: 0.9615
Epoch 336/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0457 - acc: 0.9808
Epoch 337/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1801 - acc: 0.9615
Epoch 338/500


52/52 [==============================] - 0s 364us/sample - loss: 0.0872 - acc: 0.9423
Epoch 411/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0239 - acc: 1.0000
Epoch 412/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0149 - acc: 1.0000
Epoch 413/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0560 - acc: 0.9808
Epoch 414/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0247 - acc: 1.0000
Epoch 415/500
52/52 [==============================] - 0s 365us/sample - loss: 0.1014 - acc: 0.9808
Epoch 416/500
52/52 [==============================] - 0s 365us/sample - loss: 0.0170 - acc: 1.0000
Epoch 417/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0217 - acc: 1.0000
Epoch 418/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0218 - acc: 1.0000
Epoch 419/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0420 - acc: 1.0000
Epoch 420/500


52/52 [==============================] - 0s 422us/sample - loss: 0.0197 - acc: 1.0000
Epoch 493/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0123 - acc: 1.0000
Epoch 494/500
52/52 [==============================] - 0s 441us/sample - loss: 0.0621 - acc: 0.9808
Epoch 495/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0482 - acc: 0.9808
Epoch 496/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0952 - acc: 0.9423
Epoch 497/500
52/52 [==============================] - 0s 460us/sample - loss: 0.0162 - acc: 1.0000
Epoch 498/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0143 - acc: 1.0000
Epoch 499/500
52/52 [==============================] - 0s 365us/sample - loss: 0.0181 - acc: 1.0000
Epoch 500/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0831 - acc: 0.9615


## Compile and Train Model

## Preprocess User Query

## Response Generation

## Saving unidentified Queries

# Chatbot

## Add "NA" Queries to .txt File